In [4]:
import pandas as pd
import numpy as np
import dask
import dask.dataframe as dd

import logging

from dask import distributed
import re

# with open("/home/ubuntu/work/therapeutic_accelerator/scripts/base.py") as f:
#     exec(f.read())

In [2]:
# Create embeddings function with specter model
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained('allenai/specter')
model = AutoModel.from_pretrained('allenai/specter')

from chromadb.api.types import Documents, EmbeddingFunction, Embeddings

class specter_ef(EmbeddingFunction):
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
    
    def embed_documents(self, texts: Documents) -> Embeddings:
        
        text_list = [re.sub("\n", " ", p) for p in texts]
        texts = [re.sub("\s\s+", " ", t) for t in text_list]
        
        # embed the documents somehow
        embeddings = []
        
        for text in texts:
            inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", max_length=512)
            result = model(**inputs)
            embeddings.append(result.last_hidden_state[:, 0, :])
        
        return embeddings
    
specter_embeder = specter_ef(model, tokenizer)

In [5]:
# Create dask cluster
dask.config.set(scheduler='processes')  # overwrite default with multiprocessing scheduler

cluster = distributed.LocalCluster(name='local', n_workers=7, memory_limit = '4GiB', threads_per_worker=4)  # Launches a scheduler and workers locally
client = distributed.client._get_global_client() or distributed.Client(cluster)

client

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
# import tiktoken

# @dask.delayed
def token_len(text): 
    """ Get the length of tokens from text"""
    tokens = tokenizer(text, padding=True, truncation=True, return_tensors="pt", max_length=512)['input_ids'][0]
    return len(tokens)
    
chunk_size = 2000

# create text splitters for processing the texts
text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = chunk_size,
    chunk_overlap  = 20,
    length_function = token_len
)

In [ ]:
import chromadb
from chromadb.config import Settings

# Create chroma client
chroma = chromadb.Client(Settings(chroma_api_impl="rest",
                                  chroma_server_host="54.210.84.192", # EC2 instance public IPv4
                                  chroma_server_http_port=8000))

print("Nanosecond heartbeat on server", chroma.heartbeat()) # returns a nanosecond heartbeat. Useful for making sure the client remains connected.

# Check Existing connections
chroma.list_collections()

In [ ]:
collection = chroma.get_or_create_collection("specter_abstracts", embedding_function=specter_ef(model, tokenizer))

In [ ]:
def split_text(row):
    """ Split text into chunks """
    return text_splitter.split_text(row['abstract'])

def create_doc(splited_text, corpusid):
    """ Create documents for each chunk """
        
    try:
        docs = {
            "documents": splited_text, # list of all documents [doc1, doc2, doc3, ...]
            'ids': [f'{corpusid}-{i}' for i in range(len(splited_text))], # list of all ids [id1, id2, id3, ...]
            'metadatas': [{'corpusid': int(corpusid), 'chunk': i} for i in range(len(splited_text))] # list of dictionaries with metadata for each document
        }
        return docs

    except Exception as e:
        logging.error(e)
        
def add_to_collection(docs, collection):
    """ Add documents to collection """
    
    try:
        collection.add(**docs)
        return True
    
    except Exception as e:
        logging.error(e)
        return False

In [ ]:
# Dask functions

In [ ]:
def df_main(row, collection): 
    """ Main workflow """
    
    splited_text = split_text(row)
    
    docs = create_doc(splited_text, row['corpusId'])
    
    docs['embeddings'] = specter_embeder.embed_documents(docs['documents'])[0][0].tolist()
    
    addition_results = add_to_collection(docs, collection)
    
    return docs

In [ ]:
def over_ddf(ddf, collection): 
    return ddf.apply(lambda x: x.apply(main), axis=1, meta=('x', 'object'), collection=collection)

In [ ]:
abstracts = pd.read_csv("/home/ubuntu/work/data/abstracts.csv")
abstracts.shape

In [ ]:
abstracts = abstracts.map_partitions(over_ddf, args = (collection), meta=('docs', 'object'))

In [ ]:
abstracts.head()

In [ ]:
results = abstracts.apply(main, axis=1, args=(collection,))

In [ ]:
collection.get(
    include=['documents']
)

In [ ]:
import boto3

In [ ]:
s3 = boto3.resource('s3')

# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

In [ ]:
import torch
torch.save(test, '/home/ubuntu/work/bucket/tensors_abstracts/tensor0-0.pt')

In [ ]:
# import dask processingbar
from dask.diagnostics import ProgressBar

with ProgressBar():
    tokens = df['abstract'].apply(tokenize_abstracts, meta=('abstract', 'object')).compute()

In [ ]:
tokenized = client.map(tokenize_abstracts, df['abstract'])
inputs = client.map(run_inputs, tokenized)
embeddings = client.submit(get_embeddings, inputs)

In [ ]:
# import dask processingbar
from dask.diagnostics import ProgressBar

with ProgressBar():
    abstract_embeddings = ddf['abstract'].apply(get_embeddings, meta=('abstract', 'object')).compute()